In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

from PIL import Image

In [ ]:
#upload files then unzip (colab use)
!unzip TEST.zip
!unzip TRAIN.zip

In [ ]:
# Define dataset folders
test_folder = 'TEST'
train_folder = 'TRAIN'
labels_file = 'labels.csv'

# Load labels from CSV
labels_df = pd.read_csv(labels_file)

In [19]:
# Function to preprocess images
def preprocess_images(folder_path, target_size):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.jpeg'):
            img_path = os.path.join(folder_path, filename)
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img)
            images.append(img_array)
    return np.array(images)

In [20]:
training_images = preprocess_images(train_folder, target_size=(200,200))

testing_images = preprocess_images(test_folder, target_size=(200,200))

In [24]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Load and augment training data
train_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size=(200, 200),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Load validation data
validation_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size=(200, 200),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 7968 images belonging to 4 classes.
Found 1989 images belonging to 4 classes.


In [ ]:
# Load pre-trained VGG16 model
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

# Freeze convolutional layers
for layer in vgg16.layers:
    layer.trainable = False

# Build model on top of VGG16
model = Sequential([
    vgg16,
    Flatten(),
    Dense(512, activation='relu'),
    Dense(4, activation='softmax')
])

# Compile model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

# Evaluate model
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Loss: {loss}, Validation Accuracy: {accuracy}')

Epoch 1/5
249/249 [==============================] - 4140s 17s/step - loss: 1.3727 - accuracy: 0.5016 - val_loss: 0.8959 - val_accuracy: 0.6381
Epoch 2/5
249/249 [==============================] - 4148s 17s/step - loss: 0.9046 - accuracy: 0.6258 - val_loss: 0.8010 - val_accuracy: 0.6683
Epoch 3/5
171/249 [===================>..........] - ETA: 17:06 - loss: 0.8134 - accuracy: 0.6721

In [ ]:
testing_labels = labels_df['Category'].values  # Assuming 'Category' column contains labels

# Evaluate model on testing data
testing_generator = ImageDataGenerator(rescale=1./255).flow(testing_images, testing_labels, batch_size=32)
loss, accuracy = model.evaluate(testing_generator)
print(f'Testing Loss: {loss}, Testing Accuracy: {accuracy}')